In [1]:
# --- SETUP ---
# conda install -c conda-forge google-cloud-bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
import os


c:\Users\cday\Anaconda3\lib\site-packages\geopandas\_compat.py:124: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
# import global TDM functions
import sys

sys.path.insert(0, "../Resources/2-Python/global-functions")
import BigQuery

client = BigQuery.getBigQueryClient_Confidential2023UtahHTS()


In [4]:
# --- CONFIG ---
PROJECT_ID = os.getenv("PROJECT_ID", "wfrc-modeling-data")
BQ_LOCATION = os.getenv("BQ_LOCATION", "US")
TARGET_DATASET = os.getenv("TARGET_DATASET", "prd_tdm_hts_2023")

# Optional prefix/suffix for table names
TABLE_PREFIX = os.getenv("TABLE_PREFIX", "")  # e.g., "build_20251007_"
TABLE_SUFFIX = os.getenv("TABLE_SUFFIX", "")  # e.g., "_v2"

# Ordered list of SQL files to execute
ORDERED_SQL_FILES = [
    "day.sql",
    "hh.sql",
    "person.sql",
    "trip_unlinked.sql",
    "trip_linked.sql",
    "vehicle.sql",
]

# Path to your SQL directory
SQL_DIR = ""  # adjust this path if needed

print(f"✅ Connected to BigQuery project: {PROJECT_ID}")
print(f"Target dataset: {TARGET_DATASET}")
print(f"SQL directory: {SQL_DIR}")


# --- RUNNER FUNCTION ---
def run_sql_file(sql_file: str):
    """Run a SQL file and write results to BigQuery table."""
    base_name = os.path.splitext(sql_file)[0]
    table_name = f"{TABLE_PREFIX}{base_name}{TABLE_SUFFIX}"
    table_id = f"{PROJECT_ID}.{TARGET_DATASET}.{table_name}"

    print(f"\n▶ Running: {sql_file}  →  {table_id}")

    # Load SQL from file
    with open(os.path.join(SQL_DIR, sql_file), "r", encoding="utf-8") as f:
        sql = f.read()

    # Define job config
    job_config = bigquery.QueryJobConfig(
        destination=table_id,
        write_disposition="WRITE_TRUNCATE",  # replace if exists
    )

    # Execute query
    query_job = client.query(sql, job_config=job_config)
    query_job.result()  # Wait for job to complete

    print(f"✔ Done: {table_id}")


# --- MAIN LOOP ---
for file in ORDERED_SQL_FILES:
    file_path = os.path.join(SQL_DIR, file)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Missing file: {file_path}")
    run_sql_file(file)

print("\n✅ All SQL scripts completed successfully.")


✅ Connected to BigQuery project: wfrc-modeling-data
Target dataset: prd_tdm_hts_2023
SQL directory: 

▶ Running: day.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.day
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.day

▶ Running: hh.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.hh
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.hh

▶ Running: person.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.person
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.person

▶ Running: trip_unlinked.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.trip_unlinked
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.trip_unlinked

▶ Running: trip_linked.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.trip_linked
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.trip_linked

▶ Running: vehicle.sql  →  wfrc-modeling-data.prd_tdm_hts_2023.vehicle
✔ Done: wfrc-modeling-data.prd_tdm_hts_2023.vehicle

✅ All SQL scripts completed successfully.
